In [1]:
%%capture cap --no-stderr
print("Hello World!")

In [3]:
from datetime import datetime


now = datetime.now()
filename = now.strftime("%Y-%m-%d-%H-%M-%S") + '.txt'

with open(filename, 'w') as f:
    f.write(cap.stdout)

OSError: [Errno 22] Invalid argument: '2024-06-06 21:19:33.137126.txt'